In [ ]:
import sys
import numpy
import operator
import time
import os.path

from sage.all import *

In [ ]:
# Ring LWE From Decade of Lattice Cryptography By Peikert
n = 1024;
q = 12289;
R = Integers(q)
Q = PolynomialRing(R, 'y')
S = Q.quo(Q.gen()**(n)+1, 'x')

multisub = lambda a,b: map(operator.sub, a,b)

def binom_sample():
    l = [0 for _ in range(n)]
    for i in range(n):
        b1 = [randrange(0, 2) for _ in range(BinomParam)]
        b2 = [randrange(0, 2) for _ in range(BinomParam)]
        l[i] = sum(multisub(b1,b2))
    return l

def small_sample():
    l = [0 for _ in range(n)]
    for i in range(n):
        l[i] = 1 if randrange(0, 2) == 1 else 0
    return l

a = S.random_element()
s = S(small_sample())
e = S(small_sample())
pk = (a, a * s + e)

(a, b) = pk 
r  = S(small_sample())
e  = S(small_sample())
ep = S(small_sample())

m = randrange(0,2) 
c  = (a * r + e, b * r + ep + (S(m)*floor(q/2)))

(u, v) = c 
dec = list(v - s * u)

def lwe_round(a, q):
    Temp = int(a)
    Temp1 = Temp if Temp <= floor(q/2) else Temp - q
    if Temp1 >= 0 and Temp1 <= floor(q/4):
        dec = 0
    elif Temp1 < 0 and Temp1 > - floor(q/4):
        dec = 0
    else:
        dec = 1
    return dec

res = [lwe_round(dec[i], q) for i in range(n)]

In [ ]:
def lwe_round(a, q):
    Temp = int(a)
    Temp1 = Temp if Temp <= floor(q/2) else Temp - q
    if Temp1 >= 0 and Temp1 <= floor(q/4):
        dec = 0
    elif Temp1 < 0 and Temp1 > - floor(q/4):
        dec = 0
    else:
        dec = 1
    return dec

In [ ]:
# LWE Using Library
from sage.crypto.lwe import Regev
from sage.crypto.lwe import UniformSampler
from sage.crypto.lwe import DiscreteGaussianDistributionIntegerSampler

n = 20
m = 80
lwe = Regev(n)
q = lwe.K.order()
D = DiscreteGaussianDistributionIntegerSampler(sigma=lwe.D.sigma)
sampler = UniformSampler(0,1)

def LWE_test():
    A = matrix(m, n)
    sk = vector(lwe.K, [sampler() for _ in range(n)])
    for i in range(m):
        A[i,:] = vector(lwe.K, [sampler() for _ in range(n)])
    e = vector(lwe.K, [D() for _ in range(m)])
    b = A * sk + e

    PK = (A, b)

    message = randrange(0,2)
    r = vector(lwe.K, [sampler() for _ in range(m)])
    C = (r * A, b.inner_product(r) + message * floor(q//2))

    M = C[0] * sk - C[1]
    recovered_message = lwe_round(M, q)
    
    return message == recovered_message

for i in range(100):
    if LWE_test() == False:
        print('error')
print('** Testing Done **')

In [ ]:
# RLWE Using Library
# The sigma should be looked at more closely
# The sigma from newhope paper
from sage.crypto.lwe import RingLWE
from sage.crypto.lwe import UniformSampler
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler

N = 2048
D = DiscreteGaussianDistributionPolynomialSampler(ZZ['x'], n=euler_phi(N), sigma=sqrt(8.0))
RingLWE = RingLWE(N=N, q=12289, D=D)
n = RingLWE.n
q = RingLWE.K.order()

def RingLWE_test():
    a  = RingLWE.R_q.random_element()
    sk = D()
    e  = D()
    PK = (a, a * sk + e)

    message = [randrange(0,2) for _ in range(n)]
    message_enc = RingLWE.R_q([message[i]* floor(q//2) for i in range(n)])
    r  = D()
    e  = D()
    ep = D()
    C  = (PK[0] * r + e, PK[1] * r + ep + message_enc)

    dec = list(C[1] - sk * C[0])
    recovered_message = [lwe_round(dec[i], q) for i in range(n)]
    return message == recovered_message
    
for i in range(100):
    if RingLWE_test() == False:
        print('error')
print('** Testing Done **')